In [ ]:
import itertools

In [ ]:
import h5py
import mpmath
from mpmath import mp

In [ ]:
eye3 = mp.eye(3)

def spheroid_mp(a, c, n):
    a2 = a * a
    c2_minus_a2 = c * c - a2
    return (c*c-a2)*n*n.T+a2*eye3

In [ ]:
def f_mp(lambda_, r12, q1, q2):
    one_minus_lambda = 1-lambda_
    q = one_minus_lambda*q1+lambda_*q2
    s12, _ = mp.qr_solve(q, r12)
    return lambda_*one_minus_lambda*(r12.T*s12)[0, 0]

In [ ]:
RADII = [mp.mpf(1999)/mp.mpf(100000),
         mp.mpf(1999)/mp.mpf(1000),
         mp.mpf(99999)/mp.mpf(1000)]

In [ ]:
golden_ratio = (mp.mpf(1)+mp.sqrt(mp.mpf(5)))/(mp.mpf(2))
u_abs = 1./mp.sqrt(1+golden_ratio**2)
v_abs = golden_ratio*u_abs
DIRECTIONS = []
for u in (-u_abs, u_abs):
    for v in (-v_abs, v_abs):
        DIRECTIONS += [mp.matrix((0., u, v)),
                       mp.matrix((v, 0., u)),
                       mp.matrix((u, v, 0.))]

In [ ]:
SPHEROIDS = [spheroid_mp(ai, ci, ni) for ai, ci, ni in itertools.product(RADII, RADII, DIRECTIONS)]

In [ ]:
NUM_LAMBDAS = 11
LAMBDAS = mp.linspace(0, 1, num_lambdas)

In [ ]:
n = len(SPHEROIDS)**2
for i, (q1, q2) in enumerate(itertools.product(SPHEROIDS, SPHEROIDS), 1):
    print(f'{i}/{n}')
    for r12_dir in DIRECTIONS:
        for lambda_ in LAMBDAS:
            x = f_mp(lambda_, r12_dir, q1, q2)

In [ ]:
with h5py.File('test.h5', 'w') as f:
    f['radii'] = [float(r) for r in RADII]
    f['directions'] = [(float(x), float(y), float(z)) for x, y, z in DIRECTIONS]
    f['lambdas'] = [float(val) for val in LAMBDAS]
    
    num_radii = len(RADII)
    num_dirs = len(DIRECTIONS)
    
    shape = 2*(2*(len(RADII),)+(len(DIRECTIONS),))+(len(DIRECTIONS), NUM_LAMBDAS)
    
    dset = f.create_dataset('f', shape=shape, dtype='d')
    dset[0, 0, 0, 0, 0, 0, 0] = np.linspace(0., 1., 11)